In [ ]:
# Jupyter magic for interactive plots
%matplotlib inline

In [ ]:
# Required libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

# Lesson 11: AMH Copula Parameter Estimation

The **Ali-Mikhail-Haq (AMH) copula** is an Archimedean copula that models weak to moderate dependence. Unlike the Clayton or Gumbel copulas, the AMH copula has a bounded range of dependence, making it suitable for situations where tail dependence is not expected.

**Learning Objectives:**
1. Understand the relationship between the AMH parameter ($\theta$) and Kendall's tau ($\tau$)
2. Learn to estimate $\theta$ from data using optimization
3. Visualize the $\theta$-$\tau$ relationship

---

## 1. The AMH Copula

The AMH copula is defined as:

$$ C(u, v) = \frac{uv}{1 - \theta(1-u)(1-v)} $$

where $\theta \in [-1, 1]$.

**Properties:**
- $\theta = 0$: Independence copula
- $\theta > 0$: Positive dependence
- $\theta < 0$: Negative dependence
- $\theta = 1$: Maximum positive dependence (but still moderate)

### 1.1 Kendall's Tau for the AMH Copula

The relationship between $\theta$ and Kendall's tau ($\tau$) is given by:

$$ \tau = \frac{3\theta - 2}{3\theta} - \frac{2(1-\theta)^2 \ln(1-\theta)}{3\theta^2} $$

**Reference:** Kumar, P. (2010). "A New Test for the Generalized Pareto Distribution." Applied Mathematical Sciences, 4(13-16), 757-762.
http://www.m-hikari.com/ams/ams-2010/ams-13-16-2010/kumarAMS13-16-2010.pdf

---

## 2. Forward Problem: $\theta \to \tau$

Given a copula parameter $\theta$, we can directly compute Kendall's tau using the formula above.

In [ ]:
def amh_theta_to_tau(theta):
    """
    Compute Kendall's tau from the AMH copula parameter theta.
    
    Parameters
    ----------
    theta : float or array-like
        AMH copula parameter, must be in [-1, 1)
        
    Returns
    -------
    tau : float or array-like
        Kendall's tau corresponding to the given theta
        
    Notes
    -----
    The formula is: tau = (3*theta - 2)/(3*theta) - 2*(1-theta)^2*ln(1-theta)/(3*theta^2)
    At theta = 0, tau = 0 (independence).
    """
    # Handle the special case theta = 0 (independence)
    if isinstance(theta, float):
        if theta == 0:
            return 0.0
    
    # Compute the two terms of the formula
    f1 = (3*theta - 2) / (3*theta)
    f2 = (2 * np.power(1-theta, 2) * np.log(1-theta)) / (3 * np.power(theta, 2))
    
    return f1 - f2

In [ ]:
# Create arrays of theta values for plotting
# We avoid theta = 0 (handled separately) and theta = 1 (singularity)
eps = 0.01

# Negative theta values: [-1+eps, -eps]
t1 = np.linspace(-1 + eps, -eps, 101)

# Positive theta values: [eps, 1-eps]
t2 = np.linspace(eps, 1 - eps, 101)

In [ ]:
# Visualize the theta-tau relationship
plt.figure(figsize=(8, 5))

# Plot for negative theta
plt.plot(t1, amh_theta_to_tau(t1), 'b-', linewidth=2, label=r'$\theta < 0$')

# Plot for positive theta
plt.plot(t2, amh_theta_to_tau(t2), 'b-', linewidth=2, label=r'$\theta > 0$')

# Mark the origin (independence point)
plt.scatter([0], [0], marker='o', color='k', s=100, zorder=5, label=r'Independence ($\theta=0, \tau=0$)')

plt.xlabel(r'$\theta$ (AMH parameter)', fontsize=12)
plt.ylabel(r'$\tau$ (Kendall\'s tau)', fontsize=12)
plt.title(r'AMH Copula: Relationship between $\theta$ and $\tau$', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend()
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

# Add annotations for the bounds
plt.annotate(r'$\tau_{max} \approx 1/3$', xy=(0.99, 0.33), fontsize=10)
plt.annotate(r'$\tau_{min} \approx -0.18$', xy=(-0.99, -0.18), fontsize=10)

plt.tight_layout()

---

## 3. Inverse Problem: $\tau \to \theta$

In practice, we often observe data and compute Kendall's tau empirically. We then need to find the corresponding $\theta$ value. Since there is no closed-form inverse, we use numerical optimization.

### 3.1 Objective Function

We want to find $\theta$ such that $f(\theta) = \tau$. This is equivalent to minimizing:

$$ O(\theta) = \left[ \tau - \left( \frac{3\theta - 2}{3\theta} - \frac{2(1-\theta)^2 \ln(1-\theta)}{3\theta^2} \right) \right]^2 $$

In [ ]:
def amh_tau_to_theta_objective(theta, tau):
    """
    Objective function for finding theta given tau.
    
    Returns the squared difference between the target tau and 
    the tau computed from theta.
    
    Parameters
    ----------
    theta : float
        Candidate AMH parameter
    tau : float
        Target Kendall's tau
        
    Returns
    -------
    loss : float
        Squared error (tau - amh_theta_to_tau(theta))^2
    """
    # Handle theta = 0 case
    if theta == 0:
        computed_tau = 0.0
    else:
        f1 = (3*theta - 2) / (3*theta)
        f2 = (2 * np.power(1-theta, 2) * np.log(1-theta)) / (3 * np.power(theta, 2))
        computed_tau = f1 - f2
    
    # Return squared error
    return np.power(tau - computed_tau, 2)

In [ ]:
### 3.2 Example: Estimate Theta for Positive Tau

# Target tau = 0.2 (positive dependence)
tau1 = 0.2

# Optimize over positive theta range (0, 1)
# We use bounds slightly inside (0,1) to avoid numerical issues
theta_result1 = minimize_scalar(
    amh_tau_to_theta_objective, 
    args=(tau1,), 
    method='Bounded',
    bounds=(0.001, 0.99)
)

print(f"Target tau: {tau1}")
print(f"Estimated theta: {theta_result1.x:.6f}")
print(f"Optimization result: {theta_result1}")

In [ ]:
### 3.3 Example: Estimate Theta for Negative Tau

# Target tau = -0.1 (negative dependence)
tau2 = -0.1

# Optimize over negative theta range (-1, 0)
theta_result2 = minimize_scalar(
    amh_tau_to_theta_objective, 
    args=(tau2,), 
    method='Bounded',
    bounds=(-0.99, -0.001)
)

print(f"Target tau: {tau2}")
print(f"Estimated theta: {theta_result2.x:.6f}")
print(f"Optimization result: {theta_result2}")

In [ ]:
# Visualization: Estimated Parameters on the Curve
# Visualize the estimated parameters on the theta-tau curve
plt.figure(figsize=(8, 5))

# Plot the theta-tau relationship
plt.plot(t1, amh_theta_to_tau(t1), 'b-', linewidth=2)
plt.plot(t2, amh_theta_to_tau(t2), 'b-', linewidth=2)

# Mark the origin
plt.scatter([0], [0], marker='o', color='k', s=100, zorder=5)

# Mark the estimated points
plt.scatter([theta_result1.x], [tau1], marker='*', color='r', s=200, 
            zorder=6, label=fr'$\tau={tau1}$ → $\theta={theta_result1.x:.3f}$')
plt.scatter([theta_result2.x], [tau2], marker='*', color='g', s=200, 
            zorder=6, label=fr'$\tau={tau2}$ → $\theta={theta_result2.x:.3f}$')

# Add horizontal lines from y-axis to the points
plt.hlines(y=tau1, xmin=0, xmax=theta_result1.x, colors='r', linestyles='--', alpha=0.5)
plt.hlines(y=tau2, xmin=theta_result2.x, xmax=0, colors='g', linestyles='--', alpha=0.5)

# Add vertical lines from x-axis to the points
plt.vlines(x=theta_result1.x, ymin=0, ymax=tau1, colors='r', linestyles='--', alpha=0.5)
plt.vlines(x=theta_result2.x, ymin=tau2, ymax=0, colors='g', linestyles='--', alpha=0.5)

plt.xlabel(r'$\theta$ (AMH parameter)', fontsize=12)
plt.ylabel(r'$\tau$ (Kendall\'s tau)', fontsize=12)
plt.title(r'AMH Copula: Estimating $\theta$ from $\tau$', fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(loc='upper left')
plt.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='-', alpha=0.3)

plt.tight_layout()